In [1]:
import numpy as np
from numpy import array
import string
import os
import glob
from pickle import dump, load
from time import time
from keras.models import save_model
from keras.layers import LSTM, Embedding, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.merge import add
from keras.applications.nasnet import NASNetLarge, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras import Input
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from tensorflow.keras.utils import plot_model

In [2]:
epochs = 20
batch_size=32
RANDOM_SEED=1
learning_rate=1e-3
img_size=331
np.random.seed(RANDOM_SEED)
name_suffix="_"+str(np.random.randint(1000,high=9999))
path = 'model_ckpt_NASNetLarge_bs56_half_drop'
images = r"E:\Flikr30K\flickr30k_images\flickr30k_images\\"
filename = r"E:\Flikr30K\flickr30k_images\results.txt"
model = NASNetLarge(weights='imagenet') #Load pretrained NASNetLarge model
print("Name suffix: ",name_suffix)















Name suffix:  _1235


In [3]:
# Read Caption File
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text



doc = load_doc(filename)
print(doc[:300])

image_name| comment_number| comment
1000092795.jpg| 0| Two young guys with shaggy hair look at their hands while hanging out in the yard .
1000092795.jpg| 1| Two young , White males are outside near many bushes .
1000092795.jpg| 2| Two men in green shirts are standing in a yard .
1000092795.jpg| 3| 


In [4]:
# Save Caption mapping
def load_descriptions(doc):
    mapping = dict()
    # process lines
    for line in doc.split('\n'):
        # split line by white space
        tokens = line.split("|")
        if len(line) < 3:
            continue
        # take the first token as the image id, the rest as the description
        image_id, image_desc = tokens[0], tokens[2:]
        if ".jpg" not in tokens[0]:
            continue
        # extract filename from image id
        image_id = image_id.split('.')[0]
        # convert description tokens back to string
        image_desc = ' '.join(image_desc)
        # create the list if needed
        if image_id not in mapping:
            mapping[image_id] = list()
        # store description
        mapping[image_id].append(image_desc)
    return mapping

descriptions = load_descriptions(doc)
print('Loaded: %d ' % len(descriptions))



Loaded: 31783 


In [5]:
# Preprocessing text
def clean_descriptions(descriptions):
	# prepare translation table for removing punctuation
	table = str.maketrans('', '', string.punctuation)
	for key, desc_list in descriptions.items():
		for i in range(len(desc_list)):
			desc = desc_list[i]
			# tokenize
			desc = desc.split()
			# convert to lower case
			desc = [word.lower() for word in desc]
			# remove punctuation from each token
			desc = [w.translate(table) for w in desc]
			# remove hanging 's' and 'a'
			desc = [word for word in desc if len(word)>1]
			# remove tokens with numbers in them
			desc = [word for word in desc if word.isalpha()]
			# store as string
			desc_list[i] =  ' '.join(desc)

# clean descriptions
clean_descriptions(descriptions)



In [6]:
# Save the description
def save_descriptions(descriptions, filename):
	lines = list()
	for key, desc_list in descriptions.items():
		for desc in desc_list:
			lines.append(key + ' ' + desc)
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

save_descriptions(descriptions, "descriptions"+name_suffix+".txt")



In [7]:
# Get all images
img = glob.glob(images + '*.jpg')
print(len(img))
print(img[:10])



31783
['E:\\Flikr30K\\flickr30k_images\\flickr30k_images\\1000092795.jpg', 'E:\\Flikr30K\\flickr30k_images\\flickr30k_images\\10002456.jpg', 'E:\\Flikr30K\\flickr30k_images\\flickr30k_images\\1000268201.jpg', 'E:\\Flikr30K\\flickr30k_images\\flickr30k_images\\1000344755.jpg', 'E:\\Flikr30K\\flickr30k_images\\flickr30k_images\\1000366164.jpg', 'E:\\Flikr30K\\flickr30k_images\\flickr30k_images\\1000523639.jpg', 'E:\\Flikr30K\\flickr30k_images\\flickr30k_images\\1000919630.jpg', 'E:\\Flikr30K\\flickr30k_images\\flickr30k_images\\10010052.jpg', 'E:\\Flikr30K\\flickr30k_images\\flickr30k_images\\1001465944.jpg', 'E:\\Flikr30K\\flickr30k_images\\flickr30k_images\\1001545525.jpg']


In [8]:
# Add 'startseq', 'endseq' to strings
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions

# descriptions

train_labels=[]
for i in range(len(img)):
    train_labels.append(os.path.splitext(os.path.basename(img[i]))[0])

train_descriptions = load_clean_descriptions("descriptions"+name_suffix+".txt", train_labels)
print('Descriptions: train=%d' % len(train_descriptions))

Descriptions: train=31783


In [9]:
# Process image to fit NASNet Large model.
def preprocess(image_path):
    # Convert all the images to size 331x3321 as expected by the NASNet Large model
    img = image.load_img(image_path, target_size=(img_size, img_size))
    # Convert PIL image to numpy array of 3-dimensions
    x = image.img_to_array(img)
    # Add one more dimension
    x = np.expand_dims(x, axis=0)
    # preprocess the images using preprocess_input() from nasnet module
    x = preprocess_input(x)
    return x

In [10]:
# Create a new model by popping the last layer of NASNet Large
model_new = Model(model.input, model.layers[-2].output)

In [11]:
expected_features=int(model.layers[-2].output.shape[-1])
print("Expected Feature: ",expected_features)

Expected Feature:  4032


In [12]:
# Image embedding into vector
def encode(image):
    image = preprocess(image) # preprocess the image
    fea_vec = model_new.predict(image) # Get the encoding vector for the image
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1]) # reshape from (1, 2048) to (2048, )
    return fea_vec


In [13]:
# encode image in train set
if not os.path.isfile("encoded_train_images"+name_suffix+".pkl"):
    start = time()
    encoding_train ={}
    for i in range(len(img)):
        encoding_train[os.path.basename(img[i])]=encode(img[i])
    print("Time taken in seconds =", time()-start)
    with open("encoded_train_images"+name_suffix+".pkl", "wb") as encoded_pickle:
        dump(encoding_train, encoded_pickle)


In [14]:
train_features = load(open("encoded_train_images"+name_suffix+".pkl", "rb"))
print('Photos: train=%d' % len(train_features))

Photos: train=31783


In [15]:
# List all training caption
all_train_captions = []
for key, val in train_descriptions.items():
    for cap in val:
        all_train_captions.append(cap)
len(all_train_captions)

158915

In [16]:
# Take only words that appear more than 10 times
word_count_threshold = 10
word_counts = {}
nsents = 0
for sent in all_train_captions:
    nsents += 1
    for w in sent.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1

vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]
print('preprocessed words %d -> %d' % (len(word_counts), len(vocab)))

preprocessed words 19739 -> 5437


In [17]:
ixtoword = {}
wordtoix = {}

ix = 1
for w in vocab:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1

In [18]:
vocab_size = len(ixtoword) + 1 # Add 1 for padding word
vocab_size

5438

In [19]:
# convert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

# calculate the length of the description with the most words
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)

# determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)

Description Length: 74


In [20]:
# data generator for model.fit_generator()
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            # retrieve the photo feature
            photo = photos[key+'.jpg']
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n==num_photos_per_batch:
                yield [[array(X1), array(X2)], array(y)]
                X1, X2, y = list(), list(), list()
                n=0

In [21]:
# Load Glove model
glove_dir = ''
embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.200d.txt'), encoding="utf-8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [22]:
embeddings_index['the']

array([-7.1549e-02,  9.3459e-02,  2.3738e-02, -9.0339e-02,  5.6123e-02,
        3.2547e-01, -3.9796e-01, -9.2139e-02,  6.1181e-02, -1.8950e-01,
        1.3061e-01,  1.4349e-01,  1.1479e-02,  3.8158e-01,  5.4030e-01,
       -1.4088e-01,  2.4315e-01,  2.3036e-01, -5.5339e-01,  4.8154e-02,
        4.5662e-01,  3.2338e+00,  2.0199e-02,  4.9019e-02, -1.4132e-02,
        7.6017e-02, -1.1527e-01,  2.0060e-01, -7.7657e-02,  2.4328e-01,
        1.6368e-01, -3.4118e-01, -6.6070e-02,  1.0152e-01,  3.8232e-02,
       -1.7668e-01, -8.8153e-01, -3.3895e-01, -3.5481e-02, -5.5095e-01,
       -1.6899e-02, -4.3982e-01,  3.9004e-02,  4.0447e-01, -2.5880e-01,
        6.4594e-01,  2.6641e-01,  2.8009e-01, -2.4625e-02,  6.3302e-01,
       -3.1700e-01,  1.0271e-01,  3.0886e-01,  9.7792e-02, -3.8227e-01,
        8.6552e-02,  4.7075e-02,  2.3511e-01, -3.2127e-01, -2.8538e-01,
        1.6670e-01, -4.9707e-03, -6.2714e-01, -2.4904e-01,  2.9713e-01,
        1.4379e-01, -1.2325e-01, -5.8178e-02, -1.0290e-03, -8.21

In [23]:
embedding_dim = 200

# Get 200-dim dense vector for each of the 10000 words in out vocabulary
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in wordtoix.items():
    #if i < max_words:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = embedding_vector

In [24]:
embedding_matrix.shape

(5438, 200)

In [25]:
# Create the model
inputs1 = Input(shape=(expected_features,))
fe = Dropout(0.5)(inputs1)
fe= Dense(512, activation='relu')(fe)
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(512)(se2)
decoder = add([fe, se3])
decoder = Dense(512, activation='relu')(decoder)
outputs = Dense(vocab_size, activation='softmax')(decoder)
model = Model(inputs=[inputs1, inputs2], outputs=outputs)



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [26]:
# No need to train Layer 2
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False

In [27]:
model.summary()
plot_model(model,"plot_model.png")

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 74)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 4032)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 74, 200)      1087600     input_3[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 4032)         0           input_2[0][0]                    
____________________________________________________________________________________________

In [28]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=learning_rate,decay=1e-6))

In [29]:

steps = len(train_descriptions)//batch_size


In [30]:
if not os.path.isdir(path):
    os.mkdir(path)
ckpt_cb = [ModelCheckpoint(os.path.join(path, "weights.{epoch:02d}_{loss:.3f}"+name_suffix+".h5"),
                         save_weights_only=True),
           EarlyStopping(monitor='loss', patience=3)
           ]

In [31]:
#Load previous weight if available for continue training
weigh_load=os.path.join(path,"weights.06_.h5")
if os.path.exists(weigh_load):
    model.load_weights(weigh_load)
    print("Loaded weight")

In [32]:

generator = data_generator(train_descriptions, train_features, wordtoix, max_length, batch_size)
model.fit_generator(generator,
                    epochs=epochs,
                    steps_per_epoch=steps,
                    callbacks=ckpt_cb,
                    verbose=2
                    )





Epoch 1/20


KeyboardInterrupt: 

In [ ]:
#Save the trained model
save_model(model,"image_caption_model_NASNetLarge_512"+name_suffix+".h5")